### Task 1: Defining a TypedDict Schema for Basic Extraction

Assume you are working on a Python-only project and need to extract information from movie reviews. You only need Type Hinting, not validation.

**Required Steps:**

A. Define a class `MovieInfo` inheriting from `TypedDict`.

B. Define two attributes: `title` (String) and `rating_out_of_five` (Integer).

C. *Conceptual:* If an LLM using this schema accidentally returned the rating as "4.5 stars" (a string), would this method prevent the code from running or signal an error at runtime, and why?

In [2]:
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

In [3]:
class MovieInfo(TypedDict):
    title: str
    rating_out_of_five: int

In [4]:
load_dotenv()

model = ChatOpenAI()

structured_model = model.with_structured_output(MovieInfo)

prompt = """
I gave the 4 out of 5 rating for movie named Thunderbolts
"""

res = structured_model.invoke(prompt)
print(res)

# Since there is no type checking here so whatever LLM returns won't be a issue

C:\Users\Shivansh Gupta\OneDrive\Desktop\Data Science\My Data Science\Langchain-Tutorials\langchain\Lib\site-packages\langchain_openai\chat_models\base.py:1927: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


{'title': 'Thunderbolts', 'rating_out_of_five': 4}


### Task 2: Pydantic Validation and Constraint Enforcement (Pydantic Output Parser)

You need to extract technical details about a new CPU. The extracted data must be strictly validated.

**Required Steps:**

A. Define a Pydantic `BaseModel` called `CPUDetails`.

C. Include attributes for `core_count` (Integer) and `base_frequency_ghz` (Float).

C. Use the `Field` function to enforce the following constraints on `core_count`: it must be greater than or equal to 4 and less than 12.

D. Write the initial setup for the Pydantic Output Parser, including retrieving the necessary `format_instructions`.

In [8]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate

class CPUDetails(BaseModel):
    name: str = Field(description="Name of the CPU used in the System")
    core_count: int = Field(description="Number of CPU cores", ge=4, lt=12)
    base_frequency_ghz: float = Field(description="Base frequency of CPU cores")

model = ChatOllama(model="deepseek-r1:1.5b")

parser = PydanticOutputParser(pydantic_object=CPUDetails)

template = PromptTemplate(
    template="Give me any CPU details being used in any Windows Computer or Laptop\n {format_inst}",
    input_variables=[],
    partial_variables={"format_inst": parser.get_format_instructions()}
)

chain = template | model | parser
res = chain.invoke(input={})
print(res)

name='Desktop PC' core_count=4 base_frequency_ghz=2.8


### Task 3: Simulating a Two-Step Agent Tool Chain (String Output Parser)

An Agent needs to first research a company and then summarize that information into three bullet points. Use a chain flow incorporating the String Output Parser.

**Required Steps:**

A. Define `prompt_template_research` (`input_variables=["company"]`).

B. Define `prompt_template_summary` (`input_variables=["research_output"]`).

C. Define the `StringOutputParser`.

D. Construct the full chain pipeline (Template $\rightarrow$ Model $\rightarrow$ Parser $\rightarrow$ Template $\rightarrow$ Model) to complete the task.

In [13]:
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

parser = StrOutputParser()

model = ChatOpenAI(model="gpt-5-nano")

research_template = PromptTemplate(
    template="Do a detailed research on {company}",
    input_variables=["company"],
)

summary_template = PromptTemplate(
    template="Summarize the following text in 3 bullet points.\n{text}",
    input_variables=["text"],
)

chain = research_template | model | parser | summary_template | model | parser
res = chain.invoke(input={"company": "Boeing"})
print(res)

- Boeing is a leading aerospace and defense company organized around Commercial Airplanes (CBA), Defense, Space & Security (BDS), and Global Services, with a large backlog, a global footprint, and significant NASA/space program involvement.

- It has recovered from the 737 MAX crisis but continues to navigate program delays (notably 777X), supply-chain and quality challenges, and ongoing regulatory/safety scrutiny, while expanding defense/space offerings and services to diversify revenue.

- The strategic outlook focuses on ramping high-demand commercial production, expanding defense/space modernization, and growing services and digital solutions, balanced by risks from supply chain volatility, macro conditions, and program execution; ESG considerations and international opportunities are central to strategy.

If you’d like a data-driven version with latest figures (revenue, backlog, deliveries, etc.), I can pull the most recent 10-K/quarterly results and summarize them.


#### Task 4: Structured Output with Pydantic for Review Analysis

Using the `with_structured_output` function (assuming an OpenAI model is used), create a system to analyze product reviews.

**Required Steps:**

A. Define a Pydantic class `ProductReview` containing:
- `product_name` (String, required).
- `date_of_review` (String, required).
- `is_verified_buyer` (Boolean, required).
- `sentiment_score` (Integer, must be between 1 and 5).

B. Write the Python code to define the LLM to use this schema directly via `with_structured_output`.

C. *Conceptual:* Explain what role the system prompt plays behind the scenes when using `with_structured_output` to guide the LLM toward the desired JSON format.